<a href="https://colab.research.google.com/github/faezeh-lbf/Probing-Persian-Language-Models/blob/main/multi_choice_Mask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [ ]:
! pip install git+https://github.com/huggingface/transformers.git
! pip install git+https://github.com/huggingface/datasets.git
from IPython.display import clear_output 
!pip install -q sentencepiece
!pip install ipywidgets
!pip install bertviz
clear_output()

In [ ]:
!pip install tqdm --upgrade


In [ ]:
# !python -c "from transformers import pipeline; print(pipeline('sentiment-analysis')('we love you'))"
# !pip3 install datasets
# !python -c "from datasets import load_dataset; print(load_dataset('squad', split='train')[0])"

In [ ]:
# @title train_model function

def train_model(model, train_dataloader, validation_dataloader):

  # Set the seed value all over the place to make this reproducible.
  seed_val = 42

  random.seed(seed_val)
  np.random.seed(seed_val)
  torch.manual_seed(seed_val)
  torch.cuda.manual_seed_all(seed_val)

  # We'll store a number of quantities such as training and validation loss, 
  # validation accuracy, and timings.
  training_stats = []

  # Measure the total training time for the whole run.
  total_t0 = time.time()

  optimizer = AdamW(model.parameters(),
                    lr = 2e-3, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                    eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                  )
  total_steps = len(train_dataloader) * epochs
  scheduler = get_linear_schedule_with_warmup(optimizer, 
                                              num_warmup_steps = 0, # Default value in run_glue.py
                                              num_training_steps = total_steps)
  criterion = torch.nn.CrossEntropyLoss() 

  # For each epoch...
  for epoch_i in range(0, epochs):
      
      # ========================================
      #               Training
      # ========================================
      
      # Perform one full pass over the training set.

      print("")
      print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
      print('Training...')

      # Measure how long the training epoch takes.
      t0 = time.time()

      # Reset the total loss for this epoch.
      total_train_loss = 0

      # Put the model into training mode. Don't be mislead--the call to 
      # `train` just changes the *mode*, it doesn't *perform* the training.
      # `dropout` and `batchnorm` layers behave differently during training
      # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
      model.train()

      # For each batch of training data...
      for step, batch in enumerate(train_dataloader):

          # Progress update every 40 batches.
          if step % 40 == 0 and not step == 0:
              # Calculate elapsed time in minutes.
              elapsed = format_time(time.time() - t0)
              
              # Report progress.
              print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

          # Unpack this training batch from our dataloader. 
          #
          # As we unpack the batch, we'll also copy each tensor to the GPU using the 
          # `to` method.
          #
          # `batch` contains three pytorch tensors:
          #   [0]: input ids 
          #   [1]: attention masks
          #   [2]: labels 

          b_input_ids =batch['input_ids'].to(device)
          # print(b_input_ids)
          b_input_mask = batch['attention_mask'].to(device)
          # # token_type_ids = torch.stack(batch['token_type_ids']).to(device)
          b_labels = batch['fixed_label'].to(device)

          # Always clear any previously calculated gradients before performing a
          # backward pass. PyTorch doesn't do this automatically because 
          # accumulating the gradients is "convenient while training RNNs". 
          # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
          model.zero_grad()        

          # Perform a forward pass (evaluate the model on this training batch).
          # In PyTorch, calling `model` will in turn call the model's `forward` 
          # function and pass down the arguments. The `forward` function is 
          # documented here: 
          # https://huggingface.co/transformers/model_doc/bert.html#bertforsequenceclassification
          # The results are returned in a results object, documented here:
          # https://huggingface.co/transformers/main_classes/output.html#transformers.modeling_outputs.SequenceClassifierOutput
          # Specifically, we'll get the loss (because we provided labels) and the
          # "logits"--the model outputs prior to activation.
          # print(b_input_ids.shape,b_labels.shape,b_input_mask.shape)
          result = model(ids=b_input_ids, 
                        mask=b_input_mask, 
                        labels=b_labels) 
          loss = criterion(result, b_labels)
          # print(loss)
          # loss = result.loss
          # logits = result.logits
          # print('-----------------loss------------')
          # print(loss)

          # Accumulate the training loss over all of the batches so that we can
          # calculate the average loss at the end. `loss` is a Tensor containing a
          # single value; the `.item()` function just returns the Python value 
          # from the tensor.
          total_train_loss += loss.item()

          # Perform a backward pass to calculate the gradients.
          # loss.requires_grad = True
          loss.backward()

          # Clip the norm of the gradients to 1.0.
          # This is to help prevent the "exploding gradients" problem.
          torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

          # Update parameters and take a step using the computed gradient.
          # The optimizer dictates the "update rule"--how the parameters are
          # modified based on their gradients, the learning rate, etc.
          optimizer.step()

          # Update the learning rate.
          scheduler.step()

      # Calculate the average loss over all of the batches.
      avg_train_loss = total_train_loss / len(train_dataloader)            
      
      # Measure how long this epoch took.
      training_time = format_time(time.time() - t0)

      print("")
      print("  Average training loss: {0:.2f}".format(avg_train_loss))
      print("  Training epcoh took: {:}".format(training_time))
          
      # ========================================
      #               Validation
      # ========================================
      # After the completion of each training epoch, measure our performance on
      # our validation set.

      print("")
      print("Running Validation...")

      t0 = time.time()

      # Put the model in evaluation mode--the dropout layers behave differently
      # during evaluation.
      model.eval()
      
      # Tracking variables 
      total_eval_accuracy = 0
      total_eval_loss = 0
      nb_eval_steps = 0

      # Evaluate data for one epoch
      for batch in validation_dataloader:
          
          # Unpack this training batch from our dataloader. 
          #
          # As we unpack the batch, we'll also copy each tensor to the GPU using 
          # the `to` method.
          #
          # `batch` contains three pytorch tensors:
          #   [0]: input ids 
          #   [1]: attention masks
          #   [2]: labels 
          # print(batch)
          b_input_ids = batch['input_ids'].to(device)
          b_input_mask = batch['attention_mask'].to(device)
          b_labels = batch['fixed_label'].to(device)
          
          # Tell pytorch not to bother with constructing the compute graph during
          # the forward pass, since this is only needed for backprop (training).
          with torch.no_grad():        

              # Forward pass, calculate logit predictions.
              # token_type_ids is the same as the "segment ids", which 
              # differentiates sentence 1 and 2 in 2-sentence tasks.
              result = model(ids=b_input_ids, 
                        mask=b_input_mask, 
                        labels=b_labels)

          # Get the loss and "logits" output by the model. The "logits" are the 
          # output values prior to applying an activation function like the 
          # softmax.
          loss = criterion(result, b_labels)
              
          # Accumulate the validation loss.
          total_eval_loss += loss.item()

          # Move logits and labels to CPU
          logits = result.detach().cpu().numpy()
          label_ids = b_labels.cpu().numpy()

          # Calculate the accuracy for this batch of test sentences, and
          # accumulate it over all batches.
          total_eval_accuracy += flat_accuracy(logits, label_ids)
          

      # Report the final accuracy for this validation run.
      avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
      print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

      # Calculate the average loss over all of the batches.
      avg_val_loss = total_eval_loss / len(validation_dataloader)
      
      # Measure how long the validation run took.
      validation_time = format_time(time.time() - t0)
      
      print("  Validation Loss: {0:.2f}".format(avg_val_loss))
      print("  Validation took: {:}".format(validation_time))

      # Record all statistics from this epoch.
      training_stats.append(
          {
              'epoch': epoch_i + 1,
              'Training Loss': avg_train_loss,
              'Valid. Loss': avg_val_loss,
              'Valid. Accur.': avg_val_accuracy,
              'Training Time': training_time,
              'Validation Time': validation_time
          }
      )

  print("")
  print("Training complete!")

  print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

  return avg_val_accuracy

## Loading the dataset

In [ ]:
from datasets import load_dataset, list_datasets, load_metric

batch_size = 32

# size
datasets = load_dataset('json', data_files='/content/size_comp_train.jsonl').shuffle()
dataset_test = load_dataset('json', data_files='/content/size_comp_dev.jsonl').shuffle()

# # num
# datasets = load_dataset('json', data_files='/content/ant-syn_train.jsonl').shuffle()
# dataset_test = load_dataset('json', data_files='/content/ant-syn_dev.jsonl').shuffle()

# age
# datasets = load_dataset('json', data_files='/content/age_comp_train.jsonl').shuffle()
# dataset_test = load_dataset('json', data_files='/content/age_comp_dev.jsonl').shuffle()

datasets = datasets['train'].train_test_split(test_size=0.1)
datasets['validation'] = datasets['test']
datasets['test'] = dataset_test['train']

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data. This can be easily done with the functions `load_dataset`.  

# Common

In [ ]:
from transformers import AutoModelForMaskedLM, AutoTokenizer


In [ ]:

# @title model checkpoint
# eng_model_checkpoint = "bert-base-uncased"
model_checkpoint = "HooshvareLab/bert-base-parsbert-uncased"
# model_checkpoint = "HooshvareLab/albert-fa-zwnj-base-v2"
# model_checkpoint = 'bert-base-multilingual-cased'
# model_checkpoint = 'HooshvareLab/roberta-fa-zwnj-base'

# roberta-fa-zwnj-base
# TODO: change Model to persian one


In [ ]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


# Bert

##Zero Shot

In [ ]:
from torch.nn import functional as F
import torch

In [ ]:
# model_checkpoint = "HooshvareLab/bert-base-parsbert-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
bertmodel = AutoModelForMaskedLM.from_pretrained(model_checkpoint, return_dict = True)
k = tokenizer.vocab_size
bertmodel.cpu()
print(k)

In [ ]:
# # model.bert.train=False
# for p in model.bert.parameters():
#   p.requires_grad=False
# for t in model.cls.parameters():
#   t.requires_grad=True
#   print(t.i)#.requires_grad)# print(w.name)

In [ ]:
# @title trans_numbers,large_num2word functions

def num2word(num):
  # if num=='-':
  #   return 'منفی '
  num = int(num)
  result = ''
  word = {1:'یک',2:'دو',3:'سه',4:'چهار',5:'پنج',6:'شش',7:'هفت',8:'هشت',9:'نه',10:'ده',11:'یازده',12:'دوازده',13:'سیزده',14:'چهارده',15:'پانزده',16:'شانزده',17:'هفده',18:'هجده',19:'نوزده',20:'بیست',30:'سی',40:'چهل',50:'پنجاه',60:'شصت',70:'هفتاد',80:'هشتاد',90:'نود',100:'صد',200:'دویست',300:'سیصد',400:'چهارصد',500:'پانصد',600:'ششصد',700:'هفتصد',800:'هشتصد',900:'نهصد'}
  if num==0:
    return 'صفر'
  if num<0:
    result = 'منفی '
    num*=-1
  if num <= 20:
    result+= word[num]
  elif num<100:
    result+=word[10*int(num/10)]
    if num%10>0:
      result+=' و ' + word[num%10]
  elif num<1000:
    result+=word[100*int(num/100)]
    tmp = num%100
    if tmp>0:
      result+=' و '+ num2word(tmp)

  return result

def large_num2word(num):
  if num[0]=='-':
    num=num[1:]
    result='منفی '
  else:
    result=''
  suffix={0:'',1:' هزار',2:' ملیون',3:' میلیارد',4:' تریلیون',5:' تریلیارد'}
  triples = [num[0:len(num)%3]] if len(num)%3!=0 else []
  triples +=[num[i:i+3] for i in range(len(num)%3, len(num), 3)]
  triples.reverse()
  result +=num2word(triples[-1])+suffix[len(triples)-1]
  for i in range(len(triples)-2,-1,-1):
    n=num2word(triples[i])
    if n!='صفر':
      result+=' و '+n+suffix[i]
  return result

def trans_numbers(num):
  num = num.replace('1','۱')
  num = num.replace('2','۲')
  num = num.replace('3','۳')
  num = num.replace('4','۴')
  num = num.replace('5','۵')
  num = num.replace('6','۶')
  num = num.replace('7','۷')
  num = num.replace('8','۸')
  num = num.replace('9','۹')
  num = num.replace('0','۰')
  return num


In [ ]:
def predic_mask(text, option1, option2, model, tokenizer):
  op1 = tokenizer.encode_plus(option1, return_tensors = "pt")['input_ids'][0][1]
  op2 = tokenizer.encode_plus(option2,  return_tensors = "pt")['input_ids'][0][1]
  text  =text.replace('[MASK]', tokenizer.mask_token)
  input = tokenizer.encode_plus(text, return_tensors = "pt")
  mask_index = torch.where(input["input_ids"][0] == tokenizer.mask_token_id)
  output = model(**input)
  logits = output.logits
  # softmax = F.softmax(logits, dim = -1)
  mask_word = logits[0, mask_index, :]
  # k = tokenizer.vocab_size
  # topk = torch.topk(mask_word, k, dim = 1)
  p1 = mask_word[0][op1]
  p2 = mask_word[0][op2]
  if p1>p2:
    return option1
  else:
    return option2
  # mask_word_prob = topk[0][0]
  # mask_word = topk[1][0]
  # op1_index = torch.where(mask_word==op1)
  # op2_index = torch.where(mask_word==op2)
  # sent1 =  text.replace(tokenizer.mask_token, tokenizer.decode(mask_word[op1_index]))
  # sent2 = text.replace(tokenizer.mask_token, tokenizer.decode(mask_word[op2_index]))
  # if op1_index[0] > op2_index[0]:
  #   predicted = option2
  #   # print(sent2)
  # else:
  #   predicted = option1
  #   # print(sent1)
  # # print(predicted)
  # return predicted

In [ ]:
import torch 
from torch.nn import functional as F
text = 'یک فرد num1 ساله [MASK]‌تر از یک فرد num2 ساله است.'
# text = 'num1 [MASK] num2'
options = ['جوان','پیر']
# predic_mask(text,'بزرگتر','کوچکتر', bertmodel,tokenizer)
eval = []
for i in range(15,46):
  eval.append([])
  for j in range(18,49):
    sent = text.replace('num1',trans_numbers(str(i)))
    sent = sent.replace('num2',trans_numbers(str(j)))
    p = predic_mask(sent,options[0],options[1],bertmodel,tokenizer)
    if p == options[0]:
      eval[i-15].append(0)
    elif p == options[1]:
      eval[i-15].append(1)
    else:
      print(p)
      raise 1

print(eval)

In [ ]:
import seaborn as sns
import matplotlib.pylab as plt

ax = sns.heatmap(eval,vmin=0,vmax=1,cbar=False,xticklabels=range(18,49),yticklabels=range(15,46),square=True)#, linewidth=0.5)
from google.colab import files
# plt.show()
plt.title('یک فرد ۱۰ ساله [جوان]‌تر از یک فرد ۲۰ ساله است.')
plt.savefig("bert_farsi_example.png")
files.download("bert_farsi_example.png") 


In [ ]:
# @title evaluate
def evaluate_zero_shot(tokeninzer, model, dataset):
  correct_answers= o1=o2=b=k= 0 
  for obj in dataset:
    # print(obj)
    text = obj['question']['stem']
    option1 = obj['question']['choices'][0]
    option2 = obj['question']['choices'][1]
    predicted = predic_mask(text, option1['text'], option2['text'], tokenizer=tokenizer, model=model)
    label = obj['label']
    # if obj['question']['choices'][label]['text'] == 'جوانتر':
    #   k+=1
    # elif obj['question']['choices'][label]['text'] == 'پیرتر':
    #   b+=1
    # if predicted == 'پیرتر':
    #   print(text.replace('[MASK]' , 'پیرتر'))
    if predicted == option1['text']:
      p_label = option1['label']
      o1 +=1
    elif predicted == option2['text']:
      p_label = option2['label']
      o2+=1
    else:
      print("sth wrong" , predicted)
    result = int(p_label == label) 
    # print(result)
    correct_answers +=result
  # print(o1, o2)
  # print('k , b: ' , k , b)
  # print(correct_answers, len(dataset)- correct_answers)
    # print(label, predicted)
  metric = correct_answers/len(dataset)
  return metric

metric = evaluate_zero_shot(tokenizer, bertmodel, datasets['test'])
print(metric, model_checkpoint)

## Fine-tuning

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM, AutoModelForMaskedLM, AutoTokenizer
from torch.nn import functional
import torch

model_checkpoint = "HooshvareLab/bert-base-parsbert-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
bertmodel = AutoModelForMaskedLM.from_pretrained(model_checkpoint, return_dict = True)

bertmodel.cuda()

In [ ]:
for p in bertmodel.bert.parameters():
  print(p.requires_grad)

In [ ]:
# امبرت: ساعت و تلویزیون/////////آینه چراغ گیتار  ساعت دوش موبایل تلویزیون سفال شانه
# توپ سکه پتو خودکار کلاه
# eng_model_checkpoint = "bert-base-uncased"
model_checkpoint = "HooshvareLab/bert-base-parsbert-uncased"
model_checkpoint = "HooshvareLab/albert-fa-zwnj-base-v2"
# model_checkpoint = 'bert-base-multilingual-cased'
model_checkpoint = 'HooshvareLab/roberta-fa-zwnj-base'
from transformers import BertTokenizer, BertForMaskedLM, AutoModelForMaskedLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
a = tokenizer.encode("کوچکتر بزرگتر  ")
# a=tokenizer.encode('علی سحر نفس بو جیغ نادر ناصر هاشم زهرا امیر کاوه حسین محمد حجت طاهر  مهسا فاطمه آیدا مهدی حسام پارسا شروین ')
for i in a:
  print(tokenizer.decode(i),i)#۱۴۰۰۲ ۹۶۵۲


In [ ]:
# @title check tokenizer
print('---------')
a=tokenizer.encode('کوچک تر')
for i in a:
  print(tokenizer.decode(i),i)#۱۴۰۰۲ ۹۶۵۲
print('---------')
a=tokenizer.encode('بزرگ تر')
for i in a:
  print(tokenizer.decode(i),i)#۱۴۰۰۲ ۹۶۵۲
print('---------')
a=tokenizer.encode('پیر تر')
for i in a:
  print(tokenizer.decode(i),i)#۱۴۰۰۲ ۹۶۵۲
print('---------')
a=tokenizer.encode('جوان تر')
for i in a:
  print(tokenizer.decode(i),i)#۱۴۰۰۲ ۹۶۵۲
print('---------')
a=tokenizer.encode('کوچکتر')
for i in a:
  print(tokenizer.decode(i),i)#۱۴۰۰۲ ۹۶۵۲
print('---------')
a=tokenizer.encode('بزرگتر')
for i in a:
  print(tokenizer.decode(i),i)#۱۴۰۰۲ ۹۶۵۲
print('---------')
a=tokenizer.encode('پیر‌تر')
for i in a:
  print(tokenizer.decode(i),i)#۱۴۰۰۲ ۹۶۵۲
print('---------')
a=tokenizer.encode('جوان‌تر')
for i in a:
  print(tokenizer.decode(i),i)#۱۴۰۰۲ ۹۶۵۲
print('---------')
a=tokenizer.encode('نبود')
for i in a:
  print(tokenizer.decode(i),i)#۱۴۰۰۲ ۹۶۵۲
print('---------')
a=tokenizer.encode('بود')
for i in a:
  print(tokenizer.decode(i),i)#۱۴۰۰۲ ۹۶۵۲
print('---------')

In [ ]:
epochs=4
device = torch.device("cuda")


In [ ]:
# @title fix label

def fix_labels(ex):
  c=ex['question']['choices']
  for i in c:
    if i['label']==0:
      if i['text']=='بود':
        ex['fixed_label']=ex['label']
      else:
        ex['fixed_label']=1-ex['label']
  ex['question']['stem']=ex['question']['stem'].replace('[MASK]',tokenizer.mask_token)
  enc=tokenizer.encode_plus(ex['question']['stem'],truncation=True,padding='max_length', max_length=64)
  ex={**ex,**enc}
  return ex


encoded_dataset = datasets.map(fix_labels)
encoded_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'fixed_label'])



In [ ]:
# from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
# import numpy as np
# def load_data(dataset):
# # torch.from_numpy(np.asarray(labels)
#   ids= torch.cat(dataset['input_ids'], dim=0)
#   msk= torch.cat(dataset['attention_mask'], dim=0)
#   lbl= torch.tensor(dataset['fixed_label'])

#   return TensorDataset(ids,msk, lbl)
  

# # b_input_ids = batch['input_ids'].to(device)
# # b_input_mask = batch['attention_mask'].to(device)
# # b_labels = batch['fixed_label'].to(device)
# # encoded_dataset['train']['input_ids'],'attention_mask','fixed_label']

In [ ]:
#@title dataset loader
from torch.utils.data import TensorDataset, random_split

# dataset = TensorDataset(input_ids, attention_masks, labels)


from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            encoded_dataset['train'],  # The training samples.
            sampler = RandomSampler( encoded_dataset['train']), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            encoded_dataset['validation'], # The validation samples.
            sampler = RandomSampler(encoded_dataset['validation']), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

test_dataloader = DataLoader(
            encoded_dataset['test'], # The validation samples.
            sampler = SequentialSampler(encoded_dataset['test']), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [ ]:
# for step, batch in enumerate(train_dataloader):
#   if step==0:

    # print(batch)

### Bert_MC-MLM

In [ ]:

import torch
from torch.nn import functional

class Bert_MC_MLM(torch.nn.Module):
  def __init__(self, bert_model, tokenizer, options=['بزرگتر','کوچک‌تر']):
    super(Bert_MC_MLM, self).__init__()
    self.bert_model = bert_model
    self.tokenizer = tokenizer
    self.options = options
  
  def forward(self, ids, mask, labels):
    # print("__________-----------------------______________")
    op1 = tokenizer.encode(self.options[0])[1]
    op2 = tokenizer.encode(self.options[1])[1]
    output = self.bert_model(ids,attention_mask = mask)
    logits = output.logits
    
    mask_index = torch.where(ids == tokenizer.mask_token_id)[1]

    softmax = functional.softmax(logits, dim = -1)
    mask_word = []
    for i in range(len(mask_index)):
      mask_word.append(softmax[i][mask_index[i]])
    mask_word =  torch.stack(mask_word)
    indices = torch.tensor([op1, op2]).to(device)
    predicted = torch.index_select(mask_word, 1, indices)
    # print(predicted[0][0],predicted[0][1])
    predicted = functional.softmax(predicted)
  
    return predicted


In [ ]:
model = Bert_MC_MLM(bertmodel,tokenizer,options=['کوچک‌تر','بزرگتر'])
model.cuda()
# clear_output()

In [ ]:
# for step, batch in enumerate(train_dataloader):
#   if step==0:
#     print(batch)
#     # print(torch.stack(batch['input_ids']))
#     # print(torch.FloatTensor(np.ndarray(batch['input_ids'])))
# # b_input_ids

In [ ]:
import gc
# del encoded_dataset
# del model
gc.collect()
torch.cuda.empty_cache()

### training loop


In [ ]:
import random
import numpy as np
from transformers import get_linear_schedule_with_warmup, AdamW
# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128
acc = train_model(model, train_dataloader, validation_dataloader)

In [ ]:
import torch
c = torch.tensor([[1 , 7 , 5 ,4],[1 , 3 , 3 ,7],[1 , 3 , 7 ,4]])
c2 = torch.tensor([[[1 , 7 , 5 ,4],[1 , 3 , 3 ,9],[1 , 3 , 7 ,4]], [[1 , 7 , 5 ,4],[1 , 3 , 3 ,2],[1 , 3 , 1 ,4]]])
sel = torch.where(c == 7)[1]
c2[0, sel, :]

## evaluation

In [ ]:
model.eval()

# Tracking variables 
total_eval_accuracy = 0
total_eval_loss = 0
# Evaluate data for one epoch
for batch in test_dataloader:

    b_input_ids = batch['input_ids'].to(device)
    b_input_mask = batch['attention_mask'].to(device)
    b_labels = batch['fixed_label'].to(device)
    with torch.no_grad():
      result = model(ids=b_input_ids, 
                    mask=b_input_mask, 
                    labels=b_labels)

    # Move logits and labels to CPU
    logits = result.detach().cpu().numpy()
    label_ids = b_labels.cpu().numpy()

    # Calculate the accuracy for this batch of test sentences, and
    # accumulate it over all batches.
    total_eval_accuracy += flat_accuracy(logits, label_ids)
    

# Report the final accuracy for this validation run.
avg_val_accuracy = total_eval_accuracy / len(test_dataloader)
print("  Accuracy on test set: {0:.2f}".format(avg_val_accuracy))



#roberta

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM, AutoModelForMaskedLM, AutoTokenizer
from torch.nn import functional
import torch

model_checkpoint = 'HooshvareLab/roberta-fa-zwnj-base'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
roberta_model = AutoModelForMaskedLM.from_pretrained(model_checkpoint, return_dict = True)

roberta_model.cuda()


In [ ]:
for p in roberta_model.roberta.parameters():
  p.requires_value=False


In [ ]:
epochs=4
device = torch.device("cuda")

In [ ]:
# @title fix label
def fix_labels(ex):
  c=ex['question']['choices']
  for i in c:
    if i['label']==0:
      if i['text']=='کوچکتر':
        ex['fixed_label']=ex['label']
      else:
        ex['fixed_label']=1-ex['label']
  ex['question']['stem']=ex['question']['stem'].replace('[MASK]', tokenizer.mask_token + ' ' + tokenizer.mask_token)
  enc=tokenizer.encode_plus(ex['question']['stem'],truncation=True,padding='max_length', max_length=64)
  
  ex={**ex,**enc}
  return ex

encoded_dataset = datasets.map(fix_labels)
encoded_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'fixed_label'])


In [ ]:
# @title dataset loader
from torch.utils.data import TensorDataset, random_split

# dataset = TensorDataset(input_ids, attention_masks, labels)


from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            encoded_dataset['train'],  # The training samples.
            sampler = RandomSampler( encoded_dataset['train']), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            encoded_dataset['validation'], # The validation samples.
            sampler = RandomSampler(encoded_dataset['validation']), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

test_dataloader = DataLoader(
            encoded_dataset['test'], # The validation samples.
            sampler = SequentialSampler(encoded_dataset['test']), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

### Roberta_MC-MLM

In [ ]:
c = torch.tensor([[1 , 4 , 1 , 4 ], [1 , 4 , 8 , 4 ], [1 , 4 , 4 , 4 ] ])
c2 = torch.tensor([[1 , 4 , 1 , 4 ], [1 , 4 , 8 , 4 ], [1 , 4 , 4 , 4 ] ])
c + c2

In [ ]:
import torch

class Roberta_MC_MLM(torch.nn.Module):
  def __init__(self, model, tokenizer, options=['بزرگتر','کوچک‌تر']):
    # print("__________-----------------------______________")
    super(Roberta_MC_MLM, self).__init__()
    self.model = model
    self.tokenizer = tokenizer
    self.options = [self.tokenizer.encode(option)[1:3] for option in options]
    print (self.options)
  
  def forward(self, ids, mask, labels):
    # print("__________-----------------------______________")
    output = self.model(ids,attention_mask = mask)
    logits = output.logits

    mask_index = torch.where(ids == tokenizer.mask_token_id)[1]

    softmax = functional.softmax(logits, dim = -1)
    mask1_word = []
    mask2_word = []
    for i in range(0, len(mask_index),2):
      mask1_word.append(softmax[i//2][mask_index[i]])
      mask2_word.append(softmax[(i+1)//2][mask_index[i+1]])
    mask1_word =  torch.stack(mask1_word)
    mask2_word =  torch.stack(mask2_word)
    indices1 = torch.tensor([self.options[0][0],self.options[1][0] ]).to(device)
    indices2 = torch.tensor([self.options[0][1], self.options[1][1]]).to(device)
    predicted1 = torch.index_select(mask1_word, 1, indices1)
    predicted2 = torch.index_select(mask2_word, 1, indices2)
    # print(predicted1,predicted2)
    predicted = functional.softmax(predicted1 + predicted2)
  
    return predicted


In [ ]:

model = Roberta_MC_MLM(roberta_model,tokenizer,options=['کوچکتر','بزرگتر'])
model.cuda()
# clear_output()

In [ ]:
torch.cuda.empty_cache()

In [ ]:

import random
import numpy as np
from transformers import get_linear_schedule_with_warmup, AdamW
# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

optimizer = AdamW(model.parameters(),
                  lr = 2e-1, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)
criterion = torch.nn.CrossEntropyLoss() 

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 

        b_input_ids =batch['input_ids'].to(device)
        # print(b_input_ids)
        b_input_mask = batch['attention_mask'].to(device)
        # # token_type_ids = torch.stack(batch['token_type_ids']).to(device)
        b_labels = batch['fixed_label'].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # In PyTorch, calling `model` will in turn call the model's `forward` 
        # function and pass down the arguments. The `forward` function is 
        # documented here: 
        # https://huggingface.co/transformers/model_doc/bert.html#bertforsequenceclassification
        # The results are returned in a results object, documented here:
        # https://huggingface.co/transformers/main_classes/output.html#transformers.modeling_outputs.SequenceClassifierOutput
        # Specifically, we'll get the loss (because we provided labels) and the
        # "logits"--the model outputs prior to activation.
        # print(b_input_ids.shape,b_labels.shape,b_input_mask.shape)
        result = model(ids=b_input_ids, 
                       mask=b_input_mask, 
                       labels=b_labels) 
        loss = criterion(result, b_labels)
        # print(loss)
        # loss = result.loss
        # logits = result.logits
        # print('-----------------loss------------')
        # print(loss)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        # loss.requires_grad = True
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()
    
    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        # print(batch)
        b_input_ids = batch['input_ids'].to(device)
        b_input_mask = batch['attention_mask'].to(device)
        b_labels = batch['fixed_label'].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            result = model(ids=b_input_ids, 
                       mask=b_input_mask, 
                       labels=b_labels)

        # Get the loss and "logits" output by the model. The "logits" are the 
        # output values prior to applying an activation function like the 
        # softmax.
        loss = criterion(result, b_labels)
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = result.detach().cpu().numpy()
        label_ids = b_labels.cpu().numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


In [ ]:

## evaluation

model.eval()

# Tracking variables 
total_eval_accuracy = 0
total_eval_loss = 0
# Evaluate data for one epoch
for batch in test_dataloader:

    b_input_ids = batch['input_ids'].to(device)
    b_input_mask = batch['attention_mask'].to(device)
    b_labels = batch['fixed_label'].to(device)
    with torch.no_grad():
      result = model(ids=b_input_ids, 
                    mask=b_input_mask, 
                    labels=b_labels)

    # Move logits and labels to CPU
    logits = result.detach().cpu().numpy()
    label_ids = b_labels.cpu().numpy()

    # Calculate the accuracy for this batch of test sentences, and
    # accumulate it over all batches.
    total_eval_accuracy += flat_accuracy(logits, label_ids)
    

# Report the final accuracy for this validation run.
avg_val_accuracy = total_eval_accuracy / len(test_dataloader)
print("  Accuracy on test set: {0:.2f}".format(avg_val_accuracy))


# Breakable task

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM, AutoModelForMaskedLM, AutoTokenizer
from torch.nn import functional
import torch

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
bertmodel = AutoModelForMaskedLM.from_pretrained(model_checkpoint, return_dict = True)

bertmodel.cuda()

In [ ]:
# امبرت: ساعت و تلویزیون/////////آینه چراغ گیتار  ساعت دوش موبایل تلویزیون سفال شانه
# توپ سکه پتو خودکار کلاه
# eng_model_checkpoint = "bert-base-uncased"
# model_checkpoint = "HooshvareLab/bert-base-parsbert-uncased"
# model_checkpoint = "HooshvareLab/albert-fa-zwnj-base-v2"
# model_checkpoint = 'bert-base-multilingual-cased'
model_checkpoint = 'HooshvareLab/roberta-fa-zwnj-base'

# from transformers import BertTokenizer, BertForMaskedLM, AutoModelForMaskedLM, AutoTokenizer
# model_checkpoint = "HooshvareLab/bert-base-parsbert-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
a=tokenizer.encode('توپ","سکه", "پتو", "خودکار", "کلاه","کتاب"')
# a=tokenizer.encode('"آینه","چراغ","گیتار","شیشه","ساعت","سفال"')

for i in a:
  print(tokenizer.decode(i),i)#۱۴۰۰۲ ۹۶۵۲

In [ ]:
for p in roberta_model.roberta.parameters():
  p.requres_value=False


In [ ]:

epochs=4
device = torch.device("cuda")

In [ ]:
# @title fix label
def fix_labels(ex):
  c=ex['question']['choices']
  for i in c:
    if i['label']==0:
      if i['text']=='کوچکتر':
        ex['fixed_label']=ex['label']
      else:
        ex['fixed_label']=1-ex['label']
  ex['question']['stem']=ex['question']['stem'].replace('[MASK]', tokenizer.mask_token + ' ' + tokenizer.mask_token)
  enc=tokenizer.encode_plus(ex['question']['stem'],truncation=True,padding='max_length', max_length=64)
  
  ex={**ex,**enc}
  return ex

encoded_dataset = datasets.map(fix_labels)
encoded_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'fixed_label'])


In [ ]:
# @title dataset loader
from torch.utils.data import TensorDataset, random_split

# dataset = TensorDataset(input_ids, attention_masks, labels)


from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            encoded_dataset['train'],  # The training samples.
            sampler = RandomSampler( encoded_dataset['train']), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            encoded_dataset['validation'], # The validation samples.
            sampler = RandomSampler(encoded_dataset['validation']), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

test_dataloader = DataLoader(
            encoded_dataset['test'], # The validation samples.
            sampler = SequentialSampler(encoded_dataset['test']), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [ ]:
import torch

class Break_MC_MLM(torch.nn.Module):
  def __init__(self, model, tokenizer):
    # print("__________-----------------------______________")
    super(Roberta_MC_MLM, self).__init__()
    self.model = model
    self.tokenizer = tokenizer
    self.options = [self.tokenizer.encode(option)[1:3] for option in options]
    print (self.options)
  
  def forward(self, ids, mask, labels):
    # print("__________-----------------------______________")
    output = self.model(ids,attention_mask = mask, options)
    options = [self.tokenizer.encode(option)[1] for option in options]
    logits = output.logits

    mask_index = torch.where(ids == tokenizer.mask_token_id)[1]

    softmax = functional.softmax(logits, dim = -1)
    mask1_word = []
    mask2_word = []
    for i in range(0, len(mask_index),2):
      mask1_word.append(softmax[i//2][mask_index[i]])
      mask2_word.append(softmax[(i+1)//2][mask_index[i+1]])
    mask1_word =  torch.stack(mask1_word)
    mask2_word =  torch.stack(mask2_word)
    indices1 = torch.tensor([self.options[0][0],self.options[1][0] ]).to(device)
    indices2 = torch.tensor([self.options[0][1], self.options[1][1]]).to(device)
    predicted1 = torch.index_select(mask1_word, 1, indices1)
    predicted2 = torch.index_select(mask2_word, 1, indices2)
    # print(predicted1,predicted2)
    predicted = functional.softmax(predicted1 + predicted2)
  
    return predicted


In [ ]:
model = Roberta_MC_MLM(roberta_model,tokenizer,options=['کوچکتر','بزرگتر'])
model.cuda()
# clear_output()

In [ ]:
torch.cuda.empty_cache()

### learning loop

In [ ]:
import random
import numpy as np
from transformers import get_linear_schedule_with_warmup, AdamW
# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

optimizer = AdamW(model.parameters(),
                  lr = 2e-1, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)
criterion = torch.nn.CrossEntropyLoss() 

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 

        b_input_ids =batch['input_ids'].to(device)
        # print(b_input_ids)
        b_input_mask = batch['attention_mask'].to(device)
        # # token_type_ids = torch.stack(batch['token_type_ids']).to(device)
        b_labels = batch['fixed_label'].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # In PyTorch, calling `model` will in turn call the model's `forward` 
        # function and pass down the arguments. The `forward` function is 
        # documented here: 
        # https://huggingface.co/transformers/model_doc/bert.html#bertforsequenceclassification
        # The results are returned in a results object, documented here:
        # https://huggingface.co/transformers/main_classes/output.html#transformers.modeling_outputs.SequenceClassifierOutput
        # Specifically, we'll get the loss (because we provided labels) and the
        # "logits"--the model outputs prior to activation.
        # print(b_input_ids.shape,b_labels.shape,b_input_mask.shape)
        result = model(ids=b_input_ids, 
                       mask=b_input_mask, 
                       labels=b_labels) 
        loss = criterion(result, b_labels)
        # print(loss)
        # loss = result.loss
        # logits = result.logits
        # print('-----------------loss------------')
        # print(loss)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        # loss.requires_grad = True
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()
    
    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        # print(batch)
        b_input_ids = batch['input_ids'].to(device)
        b_input_mask = batch['attention_mask'].to(device)
        b_labels = batch['fixed_label'].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            result = model(ids=b_input_ids, 
                       mask=b_input_mask, 
                       labels=b_labels)

        # Get the loss and "logits" output by the model. The "logits" are the 
        # output values prior to applying an activation function like the 
        # softmax.
        loss = criterion(result, b_labels)
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = result.detach().cpu().numpy()
        label_ids = b_labels.cpu().numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


### evaluation

In [ ]:
model.eval()

# Tracking variables 
total_eval_accuracy = 0
total_eval_loss = 0
# Evaluate data for one epoch
for batch in test_dataloader:

    b_input_ids = batch['input_ids'].to(device)
    b_input_mask = batch['attention_mask'].to(device)
    b_labels = batch['fixed_label'].to(device)
    with torch.no_grad():
      result = model(ids=b_input_ids, 
                    mask=b_input_mask, 
                    labels=b_labels)

    # Move logits and labels to CPU
    logits = result.detach().cpu().numpy()
    label_ids = b_labels.cpu().numpy()

    # Calculate the accuracy for this batch of test sentences, and
    # accumulate it over all batches.
    total_eval_accuracy += flat_accuracy(logits, label_ids)
    

# Report the final accuracy for this validation run.
avg_val_accuracy = total_eval_accuracy / len(test_dataloader)
print("  Accuracy on test set: {0:.2f}".format(avg_val_accuracy))



# multiple choice task

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model to a multiple choice task, which is the task of selecting the most plausible inputs in a given selection. The dataset used here is [SWAG](https://www.aclweb.org/anthology/D18-1009/) but you can adapt the pre-processing to any other multiple choice dataset you like, or your own data. SWAG is a dataset about commonsense reasoning, where each example describes a situation then proposes four options that could go after it. 

This notebook is built to run  with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a version with a mutiple choice head. Depending on you model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set those two parameters, then the rest of the notebook should run smoothly:

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [ ]:
from transformers import AutoTokenizer
# TODO: change tokenizer to persian one
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

We pass along `use_fast=True` to the call above to use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, but if you got an error with the previous call, remove that argument.

You can directly call this tokenizer on one sentence or a pair of sentences:

In [ ]:
test = tokenizer("این یه ۱۴ جمله فارسی است.", "با یه جمله که ادامه‌ش میاد")
tokenizer.decode(test["input_ids"])

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

To preprocess our dataset, we will thus need the names of the columns containing the sentence(s). The following dictionary keeps track of the correspondence task to column names:

We can then write the function that will preprocess our samples. The tricky part is to put all the possible pairs of sentences in two big lists before passing them to the tokenizer, then un-flatten the result so that each example has four input ids, attentions masks, etc.

When calling the `tokenizer`, we use the argument `truncation=True`. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model.

In [ ]:
# Pre process for our model

def preprocess_function(examples):
    question_headers = examples["question"]
    sentences = [[sent['stem'].replace("[MASK]", choice['text']) for choice in examples['question'][i]['choices']] for i, sent in enumerate(question_headers)]

    # Flatten everything
    sentences = sum(sentences, [])
    # for s in sentences:
    #   print(s)
    
    # Tokenize
    tokenized_examples = tokenizer(sentences, truncation=True)
    # Un-flatten
    # TODO: be warn that 2 is constant and stands for number of choices
    return {k: [v[i:i+2] for i in range(0, len(v), 2)] for k, v in tokenized_examples.items()}

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists of lists for each key: a list of all examples (here 5), then a list of all choices (4) and a list of input IDs (length varying here since we did not apply any padding):

In [ ]:
examples = datasets["train"][:5]vs
features = preprocess_function(examples)
features
print(len(features["input_ids"]), len(features["input_ids"][0]), [len(x) for x in features["input_ids"][0]])

To check we didn't do anything group when grouping all possibilites then unflattening, let's have a look at the decoded inputs for a given example:

In [ ]:
idx = 2
[tokenizer.decode(features["input_ids"][idx][i]) for i in range(2)]

We can compare it to the ground truth:

In [ ]:
show_one(datasets["train"][3])

This seems alright, so we can apply this function on all the examples in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [ ]:
encoded_datasets = datasets.map(preprocess_function, batched=True)

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

In [ ]:
encoded_datasets

In [ ]:
show_random_elements(encoded_datasets["train"])

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our task is about mutliple choice, we use the `AutoModelForMultipleChoice` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us.

In [ ]:
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer

model = AutoModelForMultipleChoice.from_pretrained(model_checkpoint)

The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `Trainer`, we will need to define three more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [ ]:
args = TrainingArguments(
    "test-glue",
    evaluation_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=4,
    weight_decay=0.01,
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay.

Then we need to tell our `Trainer` how to form batches from the pre-processed inputs. We haven't done any padding yet because we will pad each batch to the maximum length inside the batch (instead of doing so with the maximum length of the whole dataset). This will be the job of the *data collator*. A data collator takes a list of examples and converts them to a batch (by, in our case, applying padding). Since there is no data collator in the library that works on our specific problem, we will write one, adapted from the `DataCollatorWithPadding`:

In [ ]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch

@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        if "label" not in features[0].keys():
          print(features[0].keys())
          print(features[0])
          features[0]['label'] = 0
          print("----------------------------------------")
        
        # print(features[0]['label'])

        label_name = "label" if "label" in features[0].keys() else "labels"
        # print(features[0].keys())
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [[{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features]
        flattened_features = sum(flattened_features, [])
        
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        
        # Un-flatten
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        # Add back labels
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

When called on a list of examples, it will flatten all the inputs/attentions masks etc. in big lists that it will pass to the `tokenizer.pad` method. This will return a dictionary with big tensors (of shape `(batch_size * 4) x seq_length`) that we then unflatten.

We can check this data collator works on a list of features, we just have to make sure to remove all features that are not inputs accepted by our model (something the `Trainer` will do automatically for us after):

In [ ]:
accepted_keys = ["input_ids", "attention_mask", "label"]
features = [{k: v for k, v in encoded_datasets["train"][i].items() if k in accepted_keys} for i in range(0, 100, 5)]
batch = DataCollatorForMultipleChoice(tokenizer)(features)

In [ ]:
features

Again, all those flatten/un-flatten are sources of potential errors so let's make another sanity check on our inputs:

In [ ]:
[tokenizer.decode(batch["input_ids"][8][i].tolist()) for i in range(2)]

In [ ]:
show_one(datasets["train"][8])

gggtrainer.evaluate()trainer.evaluate()trainer.evaluate()trainer.evaluate()trainer.evaluate()trainer.evaluate()trainer.evaluate()trainer.evaluate()trainer.evaluate()trainer.evaluate()trainer.evaluate()trainer.evaluate()trainer.evaluate()trainer.evaluate()trainer.evaluate()trainer.evaluate()trainer.evaluate()trainer.evaluate()trainer.evaluate()trainer.evaluate()trainer.evaluate()trainer.evaluate()trainer.evaluate()trainer.evaluate()trainer.evaluate()trainer.evaluate()trainer.evaluate()trainer.evaluate()trainer.evaluate()trainer.evaluate()trainer.evaluate()trainer.evaluate()trainer.evaluate()trainer.evaluate()trainer.evaluate()trainer.evaluate()trainer.evaluate()tradfffffffieidodsifninififjidtraintrainfkffkAll good!

The last thing to define for our `Trainer` is how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier, the only preprocessing we have to do is to take the argmax of our predicted logits:

In [ ]:
import numpy as np

def compute_metrics(eval_predictions):
    predictions, label_ids = eval_predictions
    preds = np.argmax(predictions, axis=1)
    return {"accuracy": (preds == label_ids).astype(np.float32).mean().item()}

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_datasets["train"],
    eval_dataset=encoded_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer),
    compute_metrics=compute_metrics,
)

We can now finetune our model by just calling the `train` method:

In [ ]:
trainer.train()

Don't forget to [upload your model](https://huggingface.co/transformers/model_sharing.html) on the [🤗 Model Hub](https://huggingface.co/models).

In [ ]:
# inputs = tokenizer.encode("یک فرد ۱۲ ساله از یک فرد ۱۹ ساله بزرگ‌تر است", return_tensors='pt')
# outputs = model(encoded_datasets["validation"])
predict = trainer.predict(encoded_datasets['test'])
predict.metrics
# attention = outputs[-1]  # Output includes attention weights when output_attentions=True
# tokens = tokenizer.convert_ids_to_tokens(inputs[0]) 

In [ ]:
from bertviz import head_view
head_view(attention, tokens)


# TODO
<font dir='rtl'>

* چک کنم که وقتی مسک جای متفاوتی باشه هم درست کار می‌کنه؟


* برای مدل هایی که چند تا توکن می‌کنن یه کاری کنیم
*learning curve
*test datasets


# Notes


*   List item
*   List item



# Trash


In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])
# show_random_elements(eng_datasets["train"])
batch_size = 32

In [ ]:
datasets["train"]

In [ ]:
def show_one(example):
    print(f"Context: {example['question']['stem'].replace('[MASK]' , '------')}")
    print(f"  {example['question']['choices'][0]['label']}  - {example['question']['choices'][0]['text']} ")
    print(f"  {example['question']['choices'][1]['label']}  - {example['question']['choices'][1]['text']} ")

In [ ]:
show_one(datasets["train"][0])

In [ ]:


show_one(datasets["train"][15])

In [ ]:

# younger = tokenizer.encode_plus('جوانتر', return_tensors = "pt")['input_ids'][0][1]
# print(tokenizer.encode_plus('جوانتر', return_tensors = "pt")['input_ids'][0])
# older = tokenizer.encode_plus('پیرتر',  return_tensors = "pt")['input_ids'][0][1]
# print(tokenizer.encode_plus('پیرتر',  return_tensors = "pt")['input_ids'][0])
# text = "یک فرد ۲۲ ساله از یک فرد ۲ ساله [MASK] است."
# text  =text.replace('[MASK]', tokenizer.mask_token)
# input = tokenizer.encode_plus(text, return_tensors = "pt")
# mask_index = torch.where(input["input_ids"][0] == tokenizer.mask_token_id)
# print(mask_index)
# output = model(**input)
# logits = output.logits
# softmax = F.softmax(logits, dim = -1)
# mask_word = softmax[0, mask_index, :]

# mask_word_prob = torch.topk(mask_word, tokenizer.vocab_size, dim = 1)[0][0]
# mask_word = torch.topk(mask_word, tokenizer.vocab_size, dim = 1)[1][0]
# younger_index = torch.where(mask_word==younger)
# print(younger_index)
# older_index = torch.where(mask_word==older)
# print(older_index)
# younger_sent =  text.replace(tokenizer.mask_token, tokenizer.decode(mask_word[younger_index]))
# print(younger_sent, mask_word_prob[younger_index])
# if older_index[0] > younger_index[0]:
#   predicted = 'کوچکتر'
# else:
#   predicted = 'بزرگتر'
# older_sent = text.replace(tokenizer.mask_token, tokenizer.decode(mask_word[older_index]))
# print(older_sent, mask_word_prob[older_index])
# print(predicted)

# Nolang for breakable

In [ ]:
from IPython.display import clear_output
! pip install git+https://github.com/huggingface/datasets.git
!pip install jsonlines
clear_output()

In [ ]:
! pip install git+https://github.com/huggingface/transformers.git
! pip install git+https://github.com/huggingface/datasets.git
from IPython.display import clear_output 
!pip install -q sentencepiece

clear_output()
from transformers import BertTokenizer, BertForMaskedLM, AutoModelForMaskedLM, AutoTokenizer


In [ ]:
# from torch.nn import functional
# import torch
# امبرت: ساعت و تلویزیون/////////آینه چراغ گیتار شیشه  ساعت دوش موبایل تلویزیون سفال شانه
# توپ سکه پتو خودکار کلاه
# a=tokenizer.encode('علی سحر نفس بو جیغ نادر ناصر هاشم زهرا امیر کاوه حسین محمد حجت طاهر  مهسا فاطمه آیدا مهدی حسام پارسا شروین ')
# 
# eng_model_checkpoint = "bert-base-uncased"
# model_checkpoint = "HooshvareLab/bert-base-parsbert-uncased"
# model_checkpoint = "HooshvareLab/albert-fa-zwnj-base-v2"
model_checkpoint = 'bert-base-multilingual-cased'
# model_checkpoint = 'HooshvareLab/roberta-fa-zwnj-base'

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

a=tokenizer.encode('کتاب')
for i in a:
  print(tokenizer.decode(i),i)#۱۴۰۰۲ ۹۶۵۲

In [ ]:
from itertools import permutations

nbr=["محمد", "حجت", "طاهر",  "مهسا", "فاطمه", "آیدا" ]
br=["ناصر", "هاشم", "زهرا", "امیر" ,"کاوه" ,"حسین"]
dataset=[]
perm = permutations(nbr,3)
for item1,item2, item3 in iter(perm):
  for answer in br:
    for i in range(4):
      opts=[item1,item2, item3]
      opts.insert(i,answer)
      stem="مه‌رخ یک "+opts[0]+"، یک "+opts[1]+"، یک "+opts[2]+" و یک "+opts[3]+" را از پنجره پایین می‌اندازد. احتمال اینکه [MASK] بشکند از بقیه بیشتر است."
      choices=[{"label":l,"text":t}for l,t in enumerate(opts)]
      dataset.append({"question":{"stem":stem , "choices":choices},"label":i})


perm = permutations(br,3)
for item1,item2, item3 in iter(perm):
  for answer in nbr:
    for i in range(4):
      opts=[item1,item2, item3]
      opts.insert(i,answer)
      stem = "فائزه یک "+opts[0]+"، یک "+opts[1]+"، یک "+opts[2]+" و یک "+opts[3]+" را از پنجره پایین می‌اندازد. احتمال اینکه [MASK] بشکند از بقیه کمتر است."
      choices=[{"label":l,"text":t}for l,t in enumerate(opts)]
      dataset.append({"question":{"stem":stem , "choices":choices},"label":i})


In [ ]:
from IPython.display import clear_output
! pip install git+https://github.com/huggingface/datasets.git
!pip install jsonlines
clear_output()

In [ ]:
import jsonlines
with jsonlines.open("tmp.jsonl", mode="w") as writer:
  writer.write_all(dataset)


In [ ]:
dataset

In [ ]:
from datasets import load_dataset

datasets = load_dataset("json", data_files="tmp.jsonl").shuffle()
# dataset_test = load_dataset('json', data_files='/content/ant-syn_dev.jsonl').shuffle()

datasets = datasets['train'].train_test_split(760)
datasets
# datasets['validation'] = datasets['test']
# datasets['test'] = dataset_test['train']

In [ ]:
with open('breakable_train.jsonl','w') as f:
  for i in datasets['train']:
    f.write(str(i).replace("'","\"")+'\n')

with open('breakable_dev.jsonl','w') as f:
  for i in datasets['test']:
    f.write(str(i).replace("'","\"")+'\n')
    # pickle.dump(i,f)

In [ ]:
dataset_test = load_dataset('json', data_files='breakable_dev.jsonl').shuffle()


In [ ]:
dataset_test['train'][0]

In [ ]:
dataset_test['train'][0]